In [1]:
import pandas as pd
import numpy as np
import QuantLib as ql
import datetime as dt

In [21]:
df=pd.read_csv('sample_portfolio.csv')
print(df.head)

<bound method NDFrame.head of     SecurityID   IssueDate FirstSettlementDate AccrualDate DaycountBasisType  \
0            1   5/15/2020           5/15/2020   5/15/2020     Actual/Actual   
1            2    6/1/2020            6/1/2020   5/15/2020     Actual/Actual   
2            3   7/31/2019           7/31/2019   7/31/2019     Actual/Actual   
3            4   5/15/2020           5/15/2020   5/15/2020     Actual/Actual   
4            5   7/31/2020           7/31/2020   7/31/2020     Actual/Actual   
5            7   7/31/2020           7/31/2020   7/31/2020     Actual/Actual   
6            9  11/16/2020          11/16/2020  11/15/2020     Actual/Actual   
7           10    2/1/2021            2/1/2021   1/31/2021     Actual/Actual   
8           11   2/16/2021           2/16/2021   2/15/2021     Actual/Actual   
9           12   2/16/2021           2/16/2021   2/15/2021     Actual/Actual   
10          13    3/1/2021            3/1/2021   2/15/2021     Actual/Actual   
11        

In [22]:
#Need to convert PositionNotional column to float
df[' PositionNotional ']=df[' PositionNotional '].str.replace(',', '').astype(int)

In [23]:
#drop null columns
df=df.drop(['Unnamed: 10','Unnamed: 14'],axis=1)
df.head()

,SecurityID,IssueDate,FirstSettlementDate,AccrualDate,DaycountBasisType,CouponType,Coupon,FirstCouponDate,InterestPaymentFrequency,MaturityDate,Date,Price,PositionNotional
0,1,5/15/2020,5/15/2020,5/15/2020,Actual/Actual,FIX,1.500,11/15/2020,Semiannually,5/15/2050,10/7/2021,81.129,711000
1,2,6/1/2020,6/1/2020,5/15/2020,Actual/Actual,FIX,1.375,11/15/2020,Semiannually,5/15/2040,10/7/2021,85.852,589000
2,3,7/31/2019,7/31/2019,7/31/2019,Actual/Actual,FIX,2.000,1/31/2020,Semiannually,7/31/2024,10/7/2021,103.516,486000
3,4,5/15/2020,5/15/2020,5/15/2020,Actual/Actual,FIX,0.875,11/15/2020,Semiannually,5/15/2030,10/7/2021,93.153,572000
4,5,7/31/2020,7/31/2020,7/31/2020,Actual/Actual,FIX,0.500,1/31/2021,Semiannually,7/31/2025,10/7/2021,98.054,887000


In [24]:
def one_bond(row):
    """
    function to process one fixed coupon bond (Actual/Actual, Semi-annual coupons)
    ---
    input: one row from data frame
    ---
    output: YTM, Modified Duration, Dirty Price, Maturity Bucket
    """

    d=dt.datetime.strptime(row['Date'], "%m/%d/%Y")
    valuationDate = ql.Date(d.day,d.month,d.year)
    ql.Settings.instance().evaluationDate = valuationDate
    
    d=dt.datetime.strptime(row['AccrualDate'], "%m/%d/%Y")
    start = ql.Date(d.day,d.month,d.year)
    
    d=dt.datetime.strptime(row['MaturityDate'], "%m/%d/%Y")
    maturity = ql.Date(d.day,d.month,d.year)
    
    #time till maturity from valuation date
    TTM = ql.ActualActual().yearFraction(valuationDate, maturity)
    if TTM <= 5:
        mat_bucket = '5Y'
    elif TTM <= 10:
        mat_bucket = '10Y'
    elif TTM <= 20:
        mat_bucket = '20Y'
    elif TTM <= 30:
        mat_bucket = '30Y'
        
    calendar = ql.UnitedStates()
    frequency = ql.Semiannual
    tenor = ql.Period(frequency)
    dateGeneration = ql.DateGeneration.Backward
    monthEnd = True if d.day in [28,29,30,31] else False
    schedule = ql.Schedule (start, maturity, tenor, calendar, ql.Unadjusted, ql.Unadjusted, dateGeneration, monthEnd)
    daycount = ql.ActualActual(ql.ActualActual.ISMA,schedule)
    
    settlementDays = 0
    price=row['Price']
    face=100
    coupon = row['Coupon']/100
    
    bond = ql.FixedRateBond(settlementDays, face, schedule, [coupon], daycount)
    ytm = bond.bondYield(price, daycount, ql.Compounded, frequency, valuationDate,1.0e-10)
    
    rate = ql.InterestRate(ytm, daycount, ql.Compounded, frequency)
    ModDur = ql.BondFunctions.duration(bond,rate,ql.Duration.Modified)
    conv = ql.BondFunctions.convexity(bond, rate)
    dirty_p = bond.dirtyPrice(ytm, daycount, ql.Compounded, frequency)
    
    return pd.Series([ytm, ModDur, conv,dirty_p,mat_bucket],index=["YTM","Duration","Convexity","DirtyPrice","Maturity_Bucket"])



In [25]:
#apply function to calculate YTM, duration, market price and maturity bucket for all bonds in portfolio
newdf=df.apply(one_bond, axis=1)
#newdf.rename(columns={0: "YTM", 1: "Duration",2:"DirtyPrice",3:'Maturity_Bucket'},inplace=True)
newdf.head()

,YTM,Duration,Convexity,DirtyPrice,Maturity_Bucket
0,0.024176,22.214100,588.682351,81.720033,30Y
1,0.023151,15.978642,288.258546,86.393780,20Y
2,0.007356,2.733512,8.959287,103.885565,5Y
3,0.017349,8.193546,73.062372,93.497769,10Y
4,0.010213,3.760832,16.100793,98.146391,5Y


In [26]:
df=pd.concat([df,newdf],axis=1)
df.head()

,SecurityID,IssueDate,FirstSettlementDate,AccrualDate,DaycountBasisType,CouponType,Coupon,FirstCouponDate,InterestPaymentFrequency,MaturityDate,Date,Price,PositionNotional,YTM,Duration,Convexity,DirtyPrice,Maturity_Bucket
0,1,5/15/2020,5/15/2020,5/15/2020,Actual/Actual,FIX,1.500,11/15/2020,Semiannually,5/15/2050,10/7/2021,81.129,711000,0.024176,22.214100,588.682351,81.720033,30Y
1,2,6/1/2020,6/1/2020,5/15/2020,Actual/Actual,FIX,1.375,11/15/2020,Semiannually,5/15/2040,10/7/2021,85.852,589000,0.023151,15.978642,288.258546,86.393780,20Y
2,3,7/31/2019,7/31/2019,7/31/2019,Actual/Actual,FIX,2.000,1/31/2020,Semiannually,7/31/2024,10/7/2021,103.516,486000,0.007356,2.733512,8.959287,103.885565,5Y
3,4,5/15/2020,5/15/2020,5/15/2020,Actual/Actual,FIX,0.875,11/15/2020,Semiannually,5/15/2030,10/7/2021,93.153,572000,0.017349,8.193546,73.062372,93.497769,10Y
4,5,7/31/2020,7/31/2020,7/31/2020,Actual/Actual,FIX,0.500,1/31/2021,Semiannually,7/31/2025,10/7/2021,98.054,887000,0.010213,3.760832,16.100793,98.146391,5Y


In [27]:
#DV01=Modified Duration*0.01 * MarketPrice*0.01
df['DV01']=df['DirtyPrice']/100*df[' PositionNotional ']*df['Duration']*0.0001

In [28]:
#check and verify day count between any two semi-annual coupon payments is 184 
ql.ActualActual().dayCount(ql.Date(28, 2, 2021), ql.Date(31, 8, 2021))

184

In [29]:
#number of days from 10/7/2021 to 10/12/2021
ndays=ql.Date(12, 10, 2021)-ql.Date(7, 10, 2021)
ndays

5

In [30]:
#Calculate Accrued Interest on 10/7/2021
AccruedInterest_10_07=(df['DirtyPrice'] - df['Price'])/100 * df[' PositionNotional ']

#Accrued Interest on 10/12/2021
df['AccruedInterest_10.12']=AccruedInterest_10_07 + df['Coupon']/100/2 * df[' PositionNotional '] * ndays/184

In [31]:
df.head()

,SecurityID,IssueDate,FirstSettlementDate,AccrualDate,DaycountBasisType,CouponType,Coupon,FirstCouponDate,InterestPaymentFrequency,MaturityDate,Date,Price,PositionNotional,YTM,Duration,Convexity,DirtyPrice,Maturity_Bucket,DV01,AccruedInterest_10.12
0,1,5/15/2020,5/15/2020,5/15/2020,Actual/Actual,FIX,1.500,11/15/2020,Semiannually,5/15/2050,10/7/2021,81.129,711000,0.024176,22.214100,588.682351,81.720033,30Y,1290.704582,4347.146858
1,2,6/1/2020,6/1/2020,5/15/2020,Actual/Actual,FIX,1.375,11/15/2020,Semiannually,5/15/2040,10/7/2021,85.852,589000,0.023151,15.978642,288.258546,86.393780,20Y,813.088145,3301.121119
2,3,7/31/2019,7/31/2019,7/31/2019,Actual/Actual,FIX,2.000,1/31/2020,Semiannually,7/31/2024,10/7/2021,103.516,486000,0.007356,2.733512,8.959287,103.885565,5Y,138.010628,1928.152185
3,4,5/15/2020,5/15/2020,5/15/2020,Actual/Actual,FIX,0.875,11/15/2020,Semiannually,5/15/2030,10/7/2021,93.153,572000,0.017349,8.193546,73.062372,93.497769,10Y,438.196745,2040.081755
4,5,7/31/2020,7/31/2020,7/31/2020,Actual/Actual,FIX,0.500,1/31/2021,Semiannually,7/31/2025,10/7/2021,98.054,887000,0.010213,3.760832,16.100793,98.146391,5Y,327.402384,879.768910


In [32]:
#2.Aggregate DV01,PositionNotional,Accrued Interest on the maturity buckets
aggdf = df.groupby('Maturity_Bucket').agg({'DV01':'sum',' PositionNotional ':'sum','AccruedInterest_10.12':'sum'})
aggdf

,DV01,PositionNotional,AccruedInterest_10.12
Maturity_Bucket,,,
10Y,6910.999158,9717000,24605.417386
20Y,4138.454677,2676000,15042.275819
30Y,2809.330297,1389000,8395.605498
5Y,4364.430084,11547000,13386.234875


In [33]:
#3.Stress testing with duration only
port_dv01 = aggdf['DV01'].sum()
port_dv01

18223.21421601281

In [34]:
test = np.array([-25,-20,-15,-10,-5,5,10,15,20,25])
p_and_l = np.round(-test*port_dv01, 2) 
cols=[str(bps)+'bps' for bps in test]
pd.DataFrame([p_and_l],columns=cols,index=['Portfolio P&L'] )

,-25bps,-20bps,-15bps,-10bps,-5bps,5bps,10bps,15bps,20bps,25bps
Portfolio P&L,455580.36,364464.28,273348.21,182232.14,91116.07,-91116.07,-182232.14,-273348.21,-364464.28,-455580.36


In [35]:
#3. P&L with duration and convexity
def cal_p_and_l(data):
    """
    input:a row from dataframe
    output: p&l
    """
    duration = data['Duration']
    conv = data['Convexity']
    test = np.array([-25,-20,-15,-10,-5,5,10,15,20,25])
    p_and_l = (-test/10000*duration + 0.5*conv*(test/10000)**2)*data['DirtyPrice']/100*data[' PositionNotional ']
    cols=[str(bps)+'bps' for bps in test]
    
    return pd.Series(p_and_l,index=cols)

In [36]:
p_l = df.apply(cal_p_and_l, axis=1)
port_p_l = pd.DataFrame([p_l.sum(axis=0)], index=['Portfolio P&L'])
np.round(port_p_l,2)

,-25bps,-20bps,-15bps,-10bps,-5bps,5bps,10bps,15bps,20bps,25bps
Portfolio P&L,462690.11,369014.52,275907.72,183369.7,91400.46,-90831.68,-181094.58,-270788.7,-359914.04,-448470.6
